# Fashion Recommendation System
---
- Getting personal taste for fashion based on past purchases and other factors.

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gdown
from fastai.vision import *
from fastai.metrics import accuracy, top_k_accuracy
from annoy import AnnoyIndex
import zipfile
import time 
# from google.colab import drive
# from colabcode import ColabCode
%matplotlib inline

In [8]:
# get the meta data
url = 'https://drive.google.com/uc?id=0B7EVK8r0v71pWnFiNlNGTVloLUk'
output = '../data/list_category_cloth.txt'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=0B7EVK8r0v71pTGNoWkhZeVpzbFk'
output = '../data/list_category_img.txt'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=0B7EVK8r0v71pdS1FMlNreEwtc1E'
output = '../data/list_eval_partition.txt'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=0B7EVK8r0v71pWnFiNlNGTVloLUk
To: d:\ANNACONDA\projects\Projects_Challenges\personal_projects\fashion_recommendation_system\data\list_category_cloth.txt
100%|██████████| 882/882 [00:00<?, ?B/s] 
Downloading...
From: https://drive.google.com/uc?id=0B7EVK8r0v71pTGNoWkhZeVpzbFk
To: d:\ANNACONDA\projects\Projects_Challenges\personal_projects\fashion_recommendation_system\data\list_category_img.txt
100%|██████████| 21.4M/21.4M [00:00<00:00, 42.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=0B7EVK8r0v71pdS1FMlNreEwtc1E
To: d:\ANNACONDA\projects\Projects_Challenges\personal_projects\fashion_recommendation_system\data\list_eval_partition.txt
100%|██████████| 22.2M/22.2M [00:00<00:00, 38.8MB/s]


'../data/list_eval_partition.txt'

In [11]:
# get the images
root_path = './'
url = 'https://drive.google.com/uc?id=1j5fCPgh0gnY6v7ChkWlgnnHH6unxuAbb'
output = '../images/img.zip'
gdown.download(url, output, quiet=False)
with zipfile.ZipFile("img.zip","r") as zip_ref:
    zip_ref.extractall(root_path)

Downloading...
From (original): https://drive.google.com/uc?id=1j5fCPgh0gnY6v7ChkWlgnnHH6unxuAbb
From (redirected): https://drive.google.com/uc?id=1j5fCPgh0gnY6v7ChkWlgnnHH6unxuAbb&confirm=t&uuid=aa3ab323-1371-491a-a605-a3d017e7b98f
To: d:\ANNACONDA\projects\Projects_Challenges\personal_projects\fashion_recommendation_system\images\img.zip
100%|██████████| 2.68G/2.68G [01:01<00:00, 43.3MB/s]


FileNotFoundError: [Errno 2] No such file or directory: 'img.zip'

In [ ]:
category_list = []
image_path_list = []
data_type_list = []
# category names
with open('../data/list_category_cloth.txt', 'r') as f:
    for i, line in enumerate(f.readlines()):
        if i > 1:
            category_list.append(line.split(' ')[0])

# category map
with open('../data/list_category_img.txt', 'r') as f:
    for i, line in enumerate(f.readlines()):
        if i > 1:
            image_path_list.append([word.strip() for word in line.split(' ') if len(word) &gt; 0])


# train, valid, test
with open('list_eval_partition.txt', 'r') as f:
    for i, line in enumerate(f.readlines()):
        if i > 1:
            data_type_list.append([word.strip() for word in line.split(' ') if len(word) &gt; 0])

data_df = pd.DataFrame(image_path_list, columns=['image_path', 'category_number'])
data_df['category_number'] = data_df['category_number'].astype(int)
data_df = data_df.merge(pd.DataFrame(data_type_list, columns=['image_path', 'dataset_type']), on='image_path')
data_df['category'] = data_df['category_number'].apply(lambda x: category_list[int(x) - 1])
data_df = data_df.drop('category_number', axis=1)        